In [34]:
from casadi import *

In [35]:
# Set up the problem
N = 100 # number of control intervals
opti = Opti() # optimization problem

# Declare the decision variables
X = opti.variable(2, N+1) # State trajectory
pos = X[0,:]
speed = X[1,:]
U = opti.variable(1,N) # control trajectory (power) 
T = opti.variable() # final time

In [36]:
# Parameters:
mass_rider = 78
mass_bike = 8
m = opti.parameter()
opti.set_value(m, mass_rider + mass_bike)
g = 9.81
my = 0.004
b0 = 0.091
b1 = 0.0087
Iw = 0.14
r = 0.33
Cd = 0.7
rho = 1.2
A = 0.4
eta = 1
w_prime = 26630
cp = 265

In [37]:
# Set up the objective
opti.minimize(T) # race in minimal time

# System dynamics (excluding slope/gravity effect)
f = lambda x,u: vertcat(x[1], 1/x[1] * 1/(m + Iw/r**2) * (eta*u - my*m*g*x[1] - b0*x[1] - b1*x[1]**2 - 0.5*Cd*rho*A*x[1]**3)) # dx/dt = f(x,u)

In [38]:
dt = T/N # Control interval
for k in range(N): # Loop over control intervals
    k1 = f(X[:,k], U[:,k])
    k2 = f(X[:,k] + dt/2*k1, U[:,k])
    k3 = f(X[:,k] + dt/2*k2, U[:,k])
    k4 = f(X[:,k] + dt*k3, U[:,k])
    x_next = X[:,k] + dt/6*(k1+2*k2+2*k3+k4)
    opti.subject_to(X[:,k+1] == x_next)

In [39]:
# Set the path constraints
# for k in range(N): # Loop over positions
#     opti.subject_to(opti.bounded(0,pos[k],1000))

#opti.subject_to(opti.bounded(0,pos,1000))
# opti.subject_to(speed <= limit(pos)) # track speed limit
opti.subject_to(opti.bounded(1,U,500)) # control is limited

In [40]:
# Set boundary conditions
opti.subject_to(pos[0]==0) # start at position 0
#opti.subject_to(speed[0]==0.1) 
opti.subject_to(pos[-1]==1000)

opti.subject_to(speed > 0)

# One extra constraint
opti.subject_to(T>=0) # time must be positive

In [41]:
# Provide an initial guess for the solver
#opti.set_initial(T, 300)
#opti.set_initial(U, 300)

In [42]:
opti

Opti(Opti {
  instance #3
  #variables: 3 (nx = 303)
  #parameters: 1 (np = 1)
  #constraints: 105 (ng = 404)
  CasADi solver needs updating.
})

In [43]:
opti.solver('ipopt') # set numerical backend
sol = opti.solve() # actual solve

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      902
Number of nonzeros in inequality constraint Jacobian.:      202
Number of nonzeros in Lagrangian Hessian.............:      501

Error evaluating Jacobian of equality constraints at user provided starting point.
  No scaling factors for equality constraints computed!
Error evaluating Jacobian of inequality constraints at user provided starting point.
  No scaling factors for inequality constraints computed!

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total seconds in IPOPT  

CasADi - 2024-02-08 13:35:21 WARNING("solver:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 2 (row 0, col 1).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2024-02-08 13:35:21 WARNING("solver:nlp_jac_g failed: NaN detected for output jac_g_x, at nonzero index 2 (row 0, col 1).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2024-02-08 13:35:21 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2024-02-08 13:35:21 WARNING("solver:nlp_grad failed: NaN detected for output grad_gamma_p, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:377]
CasADi - 2024-02-08 13:35:21 WARNING("Failed to calculate multipliers") [.../casadi/core/nlpsol.cpp:798]


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:996: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'

In [ ]:
opti.debug.g_describe(1)

In [ ]:
opti.debug.x_describe(1)

In [ ]:
opti.debug.show_infeasibilities